### Load dataset and Create training set

In [1]:
import pandas as pd
import numpy as np
import json

from pathlib import Path
from dotenv import load_dotenv; load_dotenv()

True

In [14]:
proofwiki_raw = json.loads(Path('/Users/ton_kkrongyuth/Senior Project/Aj.Pannapa/Prove_LLM/Data/NATURALPROOFS_DATASET/naturalproofs_proofwiki.json').read_text())
# stacks_raw = json.loads(Path('/Users/ton_kkrongyuth/Senior Project/Aj.Pannapa/Prove_LLM/Data/NATURALPROOFS_DATASET/naturalproofs_stacks.json').read_text())

theorem_df = pd.DataFrame(columns=[
    'theorems_content',
    'refs_id',
    'score'
])

negative_example = pd.DataFrame(columns=[
    'theorems_content',
    'refs_id',
    'score'
])

ref_df = pd.DataFrame(columns=[
    'refs_id',
    'refs_content'
])

refference = proofwiki_raw['dataset']['definitions']
refference.extend(proofwiki_raw['dataset']['others'])

def rand_ref_idx(ref_ids:list, ref_list:list, low:int, high:int) -> int:
    rand_num = np.random.randint(low, high)
    ref_content = ref_list[rand_num - 19734]['contents']
    
    if rand_num in ref_ids or not ref_content:
        return rand_ref_idx(ref_ids, ref_list, low, high)
    else:
        return rand_num

for idx, data in enumerate(proofwiki_raw['dataset']['theorems']):
    content = ''.join(data['contents'])
    
    for ref_ids in data['ref_ids']:
        
        if ref_ids >= 19734 and ref_ids <= 33160 and content:
            theorem_df.at[idx, 'theorems_content'] = content
            theorem_df.at[idx, 'refs_id'] = ref_ids
            theorem_df.at[idx, 'score'] = float(0.9)

for idx, data in enumerate(proofwiki_raw['dataset']['theorems']):
    content = ''.join(data['contents'])
    
    for ref_ids in data['ref_ids']:
        ref_idx = rand_ref_idx(
            ref_ids=data['ref_ids'],
            ref_list=refference,
            low=19734,
            high=33160)
        
        if ref_ids >= 19734 and ref_ids <= 33160 and content:
            negative_example.at[idx, 'theorems_content'] = content
            negative_example.at[idx, 'refs_id'] = ref_idx
            negative_example.at[idx, 'score'] = float(0.3)

for idx, data in enumerate(refference):
    ref_id = data['id']
    content = ''.join(data['contents'])
    
    if ref_id and content:
        ref_df.at[idx, 'refs_id'] = ref_id
        ref_df.at[idx, 'refs_content'] = content

corpus_df = pd.merge(
    pd.concat([theorem_df, negative_example], ignore_index=True), 
    ref_df, 
    'left', 
    'refs_id'
    )[['theorems_content', 'refs_content', 'score']].dropna(ignore_index=True).sample(frac=1, ignore_index=True)

train_ratio = 0.8
train_idx = int(corpus_df.shape[0]*train_ratio)-1

corpus_df.iloc[:train_idx, :].to_csv('training_set.csv')
corpus_df.iloc[train_idx:, :].to_csv('testing_set.csv')

In [15]:
corpus_df

,theorems_content,refs_content,score
0,Let $\mathbb A$ be one of the [[Definition:Sta...,Let $S$ be a [[Definition:Set|set]].The '''res...,0.3
1,Consider the [[Definition:Order 3 Magic Square...,Magic Square/Examples/Order 305118044627842422...,0.9
2,On all the number systems::[[Definition:Natura...,'''Addition''' is the basic [[Definition:Binar...,0.9
3,"Let $T = \left({S, \tau}\right)$ be a [[Defini...","Let $T = \left({S, \tau}\right)$ be a [[Defini...",0.9
4,Let $N$ be a [[Definition:Class (Class Theory)...,Let $A$ be a [[Definition:Class (Class Theory)...,0.9
...,...,...,...
26882,"Let $T = \struct {S, \tau}$ be a [[Definition:...","Informally, the [[Definition:Set|set]] of all ...",0.3
26883,"Let $S$ be a [[Definition:String|string]], and...","Let $M = \left({A, d}\right)$ be a [[Definitio...",0.3
26884,Let $\mathcal R$ be a [[Definition:Sigma-Ring|...,A '''set''' is intuitively defined as any aggr...,0.9
26885,Let $\preccurlyeq$ be a [[Definition:Total Ord...,Let $f: S \to T$ be a [[Definition:Mapping|fun...,0.3


### Train model

In [27]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from datasets import Dataset, Value, Features

train_set = Dataset.from_csv(path_or_paths='./training_set.csv',
                             features=Features({
                                'theorems_content':Value(dtype='string'),
                                'refs_content':Value(dtype='string'),
                                'score':Value(dtype='float64')}))

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
loss = losses.CosineSimilarityLoss(model)
trining_args = SentenceTransformerTrainingArguments(
    output_dir='./temp',
    num_train_epochs=3,
)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_set,
    loss=loss,
    args=trining_args
)

trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


KeyboardInterrupt: 

### Test train model

In [ ]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from datasets import Dataset

model = SentenceTransformer("microsoft/mpnet-base")
train_dataset = Dataset.from_dict({
    "sentence1": ["It's nice weather outside today.", "He drove to work."],
    "sentence2": ["It's so sunny.", "She walked to the store."],
    "score": [1.0, 0.3],
})
loss = losses.CosineSimilarityLoss(model)

trining_args = SentenceTransformerTrainingArguments(
    output_dir='./temp',
    num_train_epochs=100,
)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    loss=loss,
    args=trining_args
)
trainer.train()

No sentence-transformers model found with name microsoft/mpnet-base. Creating a new one with mean pooling.
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.015607467889785766, metrics={'train_runtime': 23.555, 'train_samples_per_second': 8.491, 'train_steps_per_second': 4.245, 'total_flos': 0.0, 'train_loss': 0.015607467889785766, 'epoch': 100.0})

In [17]:
query = model.encode("It's nice weather outside today.")

In [18]:
document = model.encode(["It's so sunny.", "She walked to the store."])

In [15]:
from sentence_transformers.util import semantic_search

semantic_search(query_embeddings=query,
                corpus_embeddings=document)

[[{'corpus_id': 0, 'score': 0.963404655456543},
  {'corpus_id': 1, 'score': 0.6799687147140503}]]

In [19]:
from sentence_transformers.util import semantic_search

semantic_search(query_embeddings=query,
                corpus_embeddings=document)

[[{'corpus_id': 0, 'score': 0.9955084919929504},
  {'corpus_id': 1, 'score': 0.4990466237068176}]]